In [ ]:
import logging
from pathlib import Path
from Pegasus.api import *

logging.basicConfig(level = logging.DEBUG)

fa = File("input.txt")
rc = ReplicaCatalog().add_replica("local", fa, Path(".").resolve() / "input.txt")

tc = TransformationCatalog()

thumbs_up = Container(
                "thumbs_up",
                Container.DOCKER,
                image="docker://vedularaghu/thumbs_up:latest"
            )

tc.add_containers(thumbs_up)

add_thumbs = Transformation(
                        "add_thumbs",
                        site="condorpool",
                        pfn="/usr/bin/add_thumbs_up.py",
                        is_stageable=False,
                        container=thumbs_up
                )

tc.add_transformations(add_thumbs)

wf = Workflow("add_thumbs")

fb = File("output.txt")

job_add_thumbs = Job(add_thumbs)\
                    .add_args(fa, fb)\
                    .add_inputs(fa)\
                    .add_outputs(fb)

wf.add_jobs(job_add_thumbs)
wf.add_replica_catalog(rc)
wf.add_transformation_catalog(tc)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


Plan:
2020.07.09 23:35:44.568 UTC:    
2020.07.09 23:35:44.574 UTC:   ----------------------------------------------------------------------- 
2020.07.09 23:35:44.579 UTC:   File for submitting this DAG to HTCondor           : add_thumbs-0.dag.condor.sub 
2020.07.09 23:35:44.584 UTC:   Log of DAGMan debugging messages                 : add_thumbs-0.dag.dagman.out 
2020.07.09 23:35:44.590 UTC:   Log of HTCondor library output                     : add_thumbs-0.dag.lib.out 
2020.07.09 23:35:44.596 UTC:   Log of HTCondor library error messages             : add_thumbs-0.dag.lib.err 
2020.07.09 23:35:44.602 UTC:   Log of the life of condor_dagman itself          : add_thumbs-0.dag.dagman.log 
2020.07.09 23:35:44.608 UTC:    
2020.07.09 23:35:44.614 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with: 
2020.07.09 23:35:44.626 UTC:   ----------------------------------------------------------------------- 
2020.07.09 23:35:45.949 UTC:   Your database is compatible w

[######--------------------------------------------]  12.5% ..Running (Completed: 1, Queued: 0, Running: 1, Failed: 0)